# Classification and clustering using distances (Part 2)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

## $k$-nearest neighbors

$k$-nearest neighbors is an algorithm for classifying based on a notion of distance.

<font color = 'green'> __Distance/metric__: 

<font color = 'green'> __Euclidean metric__: 

In [ ]:
pets = pd.read_csv('pets.csv', index_col=0)
pets

In [ ]:
plt.plot(pets.loc[pets['label'] == 'cat', 'weight'], pets.loc[pets['label'] == 'cat', 'height'], 'o', label = 'cat')
plt.plot(pets.loc[pets['label'] == 'dog', 'weight'], pets.loc[pets['label'] == 'dog', 'height'], 's', label = 'dog')
plt.plot(pets.loc[pets['label'] == 'pony', 'weight'], pets.loc[pets['label'] == 'pony', 'height'], '^', label = 'pony')
plt.title('Pets by height, weight, class')
plt.ylabel('Height (cm)')
plt.xlabel('Weight (kg)')
plt.legend()

In [ ]:
new_pets = pd.DataFrame([[31, 30.2, 8.2], [32, 22.2, 5.0], [33, 25.8, 17.8]], columns = ['pet #','height', 'weight'])
new_pets

In [ ]:
# Drop ponies from the plot because clearly none of the new pets are ponies
plt.plot(pets.loc[pets['label'] == 'cat', 'weight'], pets.loc[pets['label'] == 'cat', 'height'], 'o', label = 'cat')
plt.plot(pets.loc[pets['label'] == 'dog', 'weight'], pets.loc[pets['label'] == 'dog', 'height'], 's', label = 'dog')
#plt.plot(pets.loc[pets['label'] == 'pony', 'weight'], pets.loc[pets['label'] == 'pony', 'height'], '^', label = 'pony')
plt.plot(new_pets['weight'], new_pets['height'], '*', label = 'unknown')
plt.title('Pets by height, weight, class')
plt.ylabel('Height (cm)')
plt.xlabel('Weight (kg)')
plt.legend()

<font color = 'green'> __Nearest neighbors:__ 

<font color = 'green'> __$k$-nearest neighbors algorithm:__ 

In [ ]:
def get_nearest_neighbors(train, instance, features, k):
    distances = pd.Series([
        sum((train.loc[i, features] - instance.loc[features]) ** 2) for i in train.index],
        index = train.index)
    indices = sorted(train.index, key = lambda i:distances[i])[:k]
    return train.loc[indices]

In [ ]:
instance = new_pets.loc[1]
features = ['weight', 'height']
instance.loc[features]

In [ ]:
get_nearest_neighbors(pets, new_pets.loc[1], features, 8)

In [ ]:
def majority_vote(nearest_neighbors):
    label = sorted(nearest_neighbors['label'], key = lambda x: sum(nearest_neighbors['label'] == x), reverse = True)[0]
    return(label)

In [ ]:
nn = get_nearest_neighbors(pets, new_pets.loc[2], features, 3)
majority_vote(nn)

In [ ]:
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
X = pets.drop('label', axis = 1)
y = pets['label']
classes = {'cat':0, 'dog':1, 'pony':2}
def make_plot(k, h):
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(X, y)
    
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X.values[:, 0].min() - 1, X.values[:, 0].max() + 1
    y_min, y_max = X.values[:, 1].min() - 1, X.values[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = np.array([classes[z] for z in clf.predict(np.c_[xx.ravel(), yy.ravel()])])

    # Put the result into a color plot
    Z = Z.reshape(yy.shape)
    plt.figure(figsize=(8,6))
    plt.pcolormesh(yy, xx, Z, cmap=cmap_light)
    plt.plot(pets.loc[pets['label'] == 'cat', 'weight'], pets.loc[pets['label'] == 'cat', 'height'], 'o', label = 'cat', color = 'black')
    plt.plot(pets.loc[pets['label'] == 'dog', 'weight'], pets.loc[pets['label'] == 'dog', 'height'], 's', label = 'dog', color = 'black')
    plt.plot(pets.loc[pets['label'] == 'pony', 'weight'], pets.loc[pets['label'] == 'pony', 'height'], '^', label = 'pony', color = 'black')
    plt.legend()
    plt.xlabel('Weight')
    plt.ylabel('Height')
    plt.title(str(k) + '-nearest neighbor classification boundaries')

for i in range(1, 8):
    make_plot(i, 0.5)

## Parameter tuning: how to choose $k$

<font color = 'green'> __Hyperparameter__: 

<font color = 'green'> __Parameter tuning__: 

<font color = 'green'> __Training and test data__: 

In [ ]:
# Data from a survey of Japanese forests
# Classes: 's': Sugi
#          'h': Hinoki
#          'd': mixed deciduous
#          'o': other non-forest land
training = pd.read_csv('training.csv', usecols=range(0,10))
y = training['class']
X = training.drop('class', axis = 1)
training

In [ ]:
test = pd.read_csv('testing.csv', usecols = range(0, 10))
test_y = test['class']
test_X = test.drop('class', axis = 1)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
clf.fit(X,y)

In [ ]:
pred = clf.predict(test_X)
confusion_matrix(test_y, pred)

<font color = 'green'> __Cross-validation__: 

<font color = 'green'> __$k$-fold cross-validation__: 

In [ ]:
# 9-fold cross-validation
folds = np.concatenate([np.repeat(i, len(y) // 9) for i in range(1, 10)])
np.random.shuffle(folds)
folds

In [ ]:
# Try different values of k, 1 through 12
mean_acc = []
for k in range(1, 13):
    acc = []
    for i in range(1, 19):
        clf = KNeighborsClassifier(n_neighbors=k, weights='uniform')
        X0 = training.loc[folds != i].drop('class', axis = 1)
        y0 = training.loc[folds != i]['class']
        X1 = training.loc[folds == i].drop('class', axis = 1)
        y1 = training.loc[folds == i]['class']
        clf.fit(X0, y0)
        acc.append(sum(clf.predict(X1) == y1) / len(y1))
    mean_acc.append(np.average(acc))
plt.plot(range(1, 20), mean_acc, 'o')

<font color = green> __Leave-one-out cross-validation__: 

In [ ]:
# Try different values of k, 1 through 20
mean_acc = []
for k in range(1, 21):
    acc = []
    for i in training.index:
        clf = KNeighborsClassifier(n_neighbors=k, weights='uniform')
        X0 = training.drop(i).drop('class', axis = 1)
        y0 = training.drop(i)['class']
        X1 = training.loc[i].drop('class')
        y1 = training.loc[i]['class']
        clf.fit(X0, y0)
        acc.append(sum(clf.predict(X1.values.reshape(1, -1)) == y1))
    mean_acc.append(np.average(acc))
plt.xticks(range(0, 21, 1))
plt.plot(range(1, 21), mean_acc, 'o')

#### Exercise

The code below loads data from various diagnostic tests of breast tissue. Each sample is classified into `car` (carcinoma), `adi` (adipose), `con` (connective), or `fmg` (fibro-adenoma, mastopathy and glandular -- we aren't concerned with distinguishing these three).

In [ ]:
tissue = pd.read_csv('tissue1.csv', index_col = 0)
tissue

Use leave-one-out cross-validation to find a good value of $k$ for classifying this data.

## Clustering

<font color = 'green'> __Supervised learning:__

<font color = 'green'> __Unsupervised learning:__

<font color = 'green'> __Clustering:__

<font color = 'green'> __Greedy algorithm:__